In [1]:
### Import libraries
import time
start_data_acquistion = time.time()
import os
import boto3
import shutil
import pathlib
import geopandas as gpd
from pathlib import Path
from datetime import datetime
from datetime import timedelta
from shapely.geometry import box, shape, Polygon
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from ipyleaflet import Map, basemaps, ScaleControl, DrawControl, SearchControl, Marker, AwesomeIcon

In [2]:
### input for path
# local path
aoi_path = ''
download_directory_path = ''

# s3 path
bucket_name = ''
drive_path = ''

### input for functiond
user = 'ab02' # mention copernicus user name
password = 'Copernicus@02' # mention password
satellite = 'sentinel-1' # input 'sentinel-1' or 'sentinel-2' for data needs to be downloaded
week_start_date = '2023-08-19' #YY-MM-DD
week_end_date = '2023-08-25' #YY-MM-DD
start_date = datetime.strptime(week_start_date, '%Y-%m-%d').date() # start date of the search period (format: YY,MM,DD)
end_date = datetime.strptime(week_end_date, '%Y-%m-%d').date() # end date of the search period (format: YY,MM,DD)
end_date = end_date + timedelta(days=1) # adding +1 day to end date to include the end date into the data search
orbit_direction =  '*' # 'ASCENDING' or 'DESCENDING' or '*' ('*' - return all product, i.e., ascending and descending)
polarisation = 'VV+VH' # input 'HH' or 'VV' or 'HV' or 'VH' or 'HH+HV' or 'VV+VH' polarization band
sensor_mode = 'IW' # input 'IW' or 'EW'
product_type = 'GRD' # input 'GRD' or 'SLC or 'S2MSI1C' (type of product for sentinel1 - 'GRD' or 'SLC' and for sentine2 - 'S2MSI1C')
cloud_coverage_percentage = 30 # mention percetage of cloud cover for the data search
relation = 'Intersects' # input 'Intersects' or 'Contains' or 'Iswithin' (Intersects -- true if the AOI and the footprint intersect (default), Contains -- true if the AOI is inside the footprint, and IsWithin -- true if the footprint is inside the AOI)
remove = False # input, i.e., True or False. if remove=True, the data used and generated from this script will be removed from local instance and if remove=False, then the data will not be removed from local instance. Deafult to False
upload = False # input, i.e., True or False. if upload=True, the data will be uploaded to s3 based on the path provided bu user and if upload=False, then the data will not be uploaded to s3. Deafult to False 
draw_poly = True # input, i.e., True or False. if draw_poly=True, the aoi will be created manually on basemap or created using the defined point and if draw_poly=False, aoi shp/geojson path is provided. Deafult to False
if draw_poly == True:
    aoi_name = input("Enter define aoi name:")
    point = input("Is the point is available for creating aoi around it, type Yes if available or type No if not available:")
    if point == 'Yes':
        latitude = input("Enter latitude of point:")
        longitude = input("Enter longitude of the point:")
        offset_degrees = input("Enter offset in degree to create bounding box around the point:") or None

In [3]:
### Functions
def create_bbox(latitude, longitude, offset_degrees=None):
    '''
    Function for creating bounding box around the lat and long based on the user input
    1. latitude: define latitude
    2. longitude: define longitude
    3. offset_degrees: define offset to create a rectangle around the lat,long

    Output:
    Return geodata frame 
    '''
    if offset_degrees == None:
        # Calculate the degree offsets for the rectangle
        offset_degrees = 0.025  # 0.025 degrees is approximately 2.75 km at the equator

    # Calculate the coordinates for the rectangle's corners
    min_lon = longitude - offset_degrees
    max_lon = longitude + offset_degrees
    min_lat = latitude - offset_degrees
    max_lat = latitude + offset_degrees

    # Create a GeoDataFrame with a single rectangular polygon
    geometry = [box(min_lon, min_lat, max_lon, max_lat)]
    gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
    return gdf

def sentinel1(api, footprint, start_date, end_date, Satellite, product_type, sensor_mode, polarisation, relation, orbit_direction, metadata_file_path, imagery_path):
    '''
    Function for querying sentinel1 data
    1. api: sentinel API
    2. start_date: start date of the search period 
    3. end_date: end date of the search period
    4. satellite: 'sentinel-1'
    5. product_type: 'S2MSI1C' (type of data provided by sentinel2)
    6. sensor_mode: 'IW' or 'EW' (type of sensor provided by sentinel1)
    7. polarisation: 'HH' or 'VV' or 'HV' or 'VH' or 'HH+HV' or 'VV+VH' (type of polarization bands offer by sentinel1)
    8. relation: 'Intersects' or 'Contains' or 'Iswithin' [Intersects: true if the AOI and the footprint intersect (default), Contains: true if the AOI is inside the footprint, and IsWithin: true if the footprint is inside the AOI]
    9. orbit_direction: 'ASCENDING' or 'DESCENDING' or '*' ('*' -- Ascendin and Descending)
    10. metadata_path: Location where the metadata csv will be saved
    11. imagery_path: Location where the imageries csv will be saved

    Output:
    Return products
    '''
    print(f'Data is downloading from Sentinel1....')
    products = api.query(footprint,
                     date = (start_date, end_date),
                     platformname = Satellite,
                     producttype = product_type,
                     sensoroperationalmode = sensor_mode,
                     polarisationmode = polarisation,
                     area_relation = relation,
                     orbitdirection = orbit_direction)
    
    # print('Total images: ', len(products)) # Number of images are available
    # GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
    products_df = api.to_dataframe(products)
    products_df.to_csv(metadata_file_path)
    
    # download all results from the search
    api.download_all(products, directory_path = imagery_path,)
    return products
    
def sentinel2(api, footprint, start_date, end_date, Satellite, product_type, cloud_coverage_percentage, relation, metadata_file_path, imagery_path):
    '''
    Function for querying sentinel2 data
    1. api: sentinel API
    2. start_date: start date of the search period 
    3. end_date: end date of the search period
    4. satellite: 'sentinel-2'
    5. colud_coverage_precentage: define percetage of cloud cover for the data search
    6. relation: 'Intersects' or 'Contains' or 'Iswithin' [Intersects: true if the AOI and the footprint intersect (default), Contains: true if the AOI is inside the footprint, and IsWithin: true if the footprint is inside the AOI]
    7. metadata_path: Location where the metadata csv will be saved
    8. imagery_path: Location where the imageries csv will be saved

    Output:
    Return products
    '''
    print(f'Data is downloading from Sentinel2....')
    products = api.query(footprint,
                     date = (start_date, end_date),
                     platformname = Satellite,
                     producttype = product_type,
                     cloudcoverpercentage = (0, cloud_coverage_percentage),
                     area_relation = relation)
    
    # print('Total images: ', len(products)) # Number of images are available
    # GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
    products_df = api.to_geodataframe(products)
    products_df.to_csv(metadata_file_path)
    
    # download all results from the search
    api.download_all(products, directory_path = imagery_path,)
    return products

def s3_upload(bucket_name, local_path, drive_path):
    '''
    Function is to upload the whole directory data to the desired location on s3
    Input parameters
    1. bucket_name: Define the name of the s3 bucket where data needs to be uploaded
    2. local_path: Location of the directory in the local instance
    3. drive_path: Location of the directory in the s3 bucket
    '''
    print(f'Data is uploading from local to s3....')
    s3 = boto3.client('s3')
    for file in os.listdir(local_path):
        if not file.startswith('.'):
            local_file_path = os.path.join(local_path, file)
            drive_file_path = os.path.join(drive_path, file)
            s3.upload_file(Filename=local_file_path, Bucket=bucket_name, Key=drive_file_path)

In [10]:
if draw_poly == True and point == 'No':
  # Create ipyleaflet map, add tile layer, and display
  center = [38.128, 2.588]
  zoom = 5

  m = Map(basemap=basemaps.Esri.WorldTopoMap, center=center, zoom=zoom)
  m.add_control(ScaleControl(position='bottomleft'))

  marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

  m.add_control(SearchControl(
    position="topright",
    url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
    zoom=5,
    marker=marker
  ))

  draw_control = DrawControl(
      marker={"shapeOptions": {"color": "#0000FF"}},
      rectangle={"shapeOptions": {"color": "#0000FF"}},
      circle={"shapeOptions": {"color": "#0000FF"}},
      circlemarker={},
  )
  m.add(draw_control)
  display(m)

Map(center=[38.128, 2.588], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [5]:
if draw_poly == True:
    if point == 'Yes':
        gdf_bounds = create_bbox(latitude, longitude)
    elif point == 'No':
        polygon = shape(draw_control.last_draw.get('geometry'))
        gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon])
        bounds = gdf.total_bounds 
        gdf_bounds = gpd.GeoSeries([box(*bounds)])

    footprint = gdf_bounds.to_wkt().values.tolist()[0]

elif draw_poly == False:
    if aoi_path.endswith('.geojson'):
        # read geojson
        footprint = geojson_to_wkt(read_geojson(aoi_path))
    elif aoi_path.endswith('.shp'):
        # read shapefile
        gdf = gpd.read_file(aoi_path)
        bounds = gdf.total_bounds
        gdf_bounds = gpd.GeoSeries([box(*bounds)])
        footprint = gdf_bounds.to_wkt().values.tolist()[0]
    aoi_name = pathlib.Path(aoi_path).stem

In [ ]:
# accessing the sentinelsat API using user name and password
api = SentinelAPI(user, password)

metadata_file_path = os.path.join(download_directory_path, f'{aoi_name}_acqusition_{week_start_date}_{week_end_date}.csv')

if satellite == 'sentinel-1':
    products = sentinel1(api, 
                            footprint, 
                            start_date, 
                            end_date, satellite, 
                            product_type, 
                            sensor_mode, 
                            polarisation, 
                            relation, 
                            orbit_direction, metadata_file_path, download_directory_path)
elif satellite == 'sentinel-2':
    products = sentinel2(api, 
                         footprint, 
                         start_date, 
                         end_date, 
                         satellite, 
                         product_type, 
                         cloud_coverage_percentage,
                         relation, metadata_file_path, download_directory_path)

    downloaded_img = [file for file in Path(download_directory_path).rglob('*.zip')]
    print('Total avaliable images: ', len(products)) # Number of images are available
    # print(downloaded_img)
    print('Total downloaded images: ', len(downloaded_img)) # Number of images downloaded

    ### Export to s3
    if upload == True:
        # Connect to S3 bucket and download file 
        s3_upload(bucket_name, download_directory_path, drive_path)
    elif upload == False:
        pass
    
    # remove the data from the local
    if remove == True:
        shutil.rmtree(download_directory_path)  # remove raw imagery folder
    elif remove == False:
        pass

    end = time.time()
    end_data_acquistion = time.time()
    print("The time of execution of data acquistion script:",(end_data_acquistion-start_data_acquistion))